In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import wrangle
import prep

wrangle.py functions loaded successfully
acquire.py functions loaded successfully


In [2]:
df = prep.prep_zillow_data()

In [3]:
X_train, y_train, X_validate, y_validate, X_test, y_test = wrangle.train_validate_test(df)

In [4]:
X_train_scaled, X_validate_scaled, X_test_scaled = wrangle.min_max_scale(X_train, X_validate, X_test)

In [5]:
X_train_scaled

,id,parcelid,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,heatingorsystemtypeid,...,regionidzip,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,censustractandblock,propertylandusetypeid,logerror
3927,0.599719,0.025732,0.272727,0.166667,0.20,0.182322,0.182322,0.297297,0.1,0.128510,...,0.003309,0.000000,0.03609,0.825503,0.040794,0.0,0.028981,0.296872,0.689655,0.633541
19745,0.627826,0.011624,0.090909,0.166667,0.00,0.043299,0.043299,0.000000,0.0,0.260870,...,0.000840,0.000000,0.00000,0.563758,0.013765,0.0,0.026478,0.006444,0.517241,0.636842
1732,0.059076,0.023485,0.227273,0.250000,0.15,0.103328,0.103328,0.297297,0.1,0.128510,...,0.003309,0.000000,0.03609,0.825503,0.016087,0.0,0.029037,0.296872,0.517241,0.631335
17739,0.045942,0.003047,0.272727,0.250000,0.20,0.125783,0.125783,0.000000,0.2,0.043478,...,0.004435,0.000000,0.00000,0.845638,0.015576,0.0,0.001744,0.010947,0.517241,0.622549
15673,0.137188,0.009659,0.090909,0.250000,0.00,0.063697,0.063697,0.000000,0.0,0.260870,...,0.000003,0.000000,0.00000,0.523490,0.002053,0.0,0.000747,0.001906,0.517241,0.623408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11371,0.041640,0.007389,0.181818,0.166667,0.10,0.063607,0.063607,0.000000,0.1,0.043478,...,0.000135,0.000000,0.00000,0.778523,0.016798,0.0,0.013253,0.001356,0.689655,0.624099
21006,0.360829,0.002385,0.272727,0.250000,0.20,0.106280,0.106280,0.000000,0.2,0.043478,...,0.001199,0.000000,0.00000,0.771812,0.030207,0.0,0.004726,0.000165,0.689655,0.631399
20936,0.538675,0.020660,0.272727,0.416667,0.20,0.197531,0.197531,0.297297,0.2,0.128510,...,0.003319,0.571429,0.03609,0.724832,0.018350,0.0,0.031033,0.297502,0.517241,0.631720
3675,0.317486,0.010941,0.090909,0.250000,0.00,0.069959,0.069959,0.000000,0.0,0.043478,...,0.000761,0.000000,0.00000,0.563758,0.010447,0.0,0.012499,0.006358,0.517241,0.625939


In [6]:
from sklearn.feature_selection import SelectKBest, f_regression

In [7]:
f_selector = SelectKBest(f_regression, k=8)

In [8]:
f_selector = f_selector.fit(X_train_scaled, y_train.taxvaluedollarcnt)

In [9]:
X_train_reduced = f_selector.transform(X_train_scaled)
print(X_train.shape)
print(X_train_reduced.shape)

(11956, 27)
(11956, 8)


In [10]:
f_support = f_selector.get_support()
f_support

array([False, False,  True,  True,  True,  True,  True, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False,  True, False, False, False])

In [11]:
f_feature = X_train_scaled.iloc[:, f_support].columns.tolist()
f_feature

['bathroomcnt',
 'bedroomcnt',
 'calculatedbathnbr',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet12',
 'fullbathcnt',
 'structuretaxvaluedollarcnt',
 'landtaxvaluedollarcnt']

In [12]:
X_reduced_scaled = X_train_scaled.iloc[:, f_support]
X_reduced_scaled.head()

,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fullbathcnt,structuretaxvaluedollarcnt,landtaxvaluedollarcnt
3927,0.272727,0.166667,0.20,0.182322,0.182322,0.1,0.040794,0.028981
19745,0.090909,0.166667,0.00,0.043299,0.043299,0.0,0.013765,0.026478
1732,0.227273,0.250000,0.15,0.103328,0.103328,0.1,0.016087,0.029037
17739,0.272727,0.250000,0.20,0.125783,0.125783,0.2,0.015576,0.001744
15673,0.090909,0.250000,0.00,0.063697,0.063697,0.0,0.002053,0.000747


In [13]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [14]:
lm = LinearRegression()

In [15]:
rfe = RFE(lm, 8)

In [16]:
X_rfe = rfe.fit_transform(X_train_scaled, y_train)

In [17]:
mask = rfe.support_

In [18]:
X_reduced_scaled_rfe = X_train_scaled.iloc[:, mask]

In [19]:
X_reduced_scaled = X_train_scaled.iloc[:, f_support]
X_reduced_scaled.head()

,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fullbathcnt,structuretaxvaluedollarcnt,landtaxvaluedollarcnt
3927,0.272727,0.166667,0.20,0.182322,0.182322,0.1,0.040794,0.028981
19745,0.090909,0.166667,0.00,0.043299,0.043299,0.0,0.013765,0.026478
1732,0.227273,0.250000,0.15,0.103328,0.103328,0.1,0.016087,0.029037
17739,0.272727,0.250000,0.20,0.125783,0.125783,0.2,0.015576,0.001744
15673,0.090909,0.250000,0.00,0.063697,0.063697,0.0,0.002053,0.000747
